In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn import datasets
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras.models import *

wine = datasets.load_wine()
comb = np.c_[wine.data, wine.target]
df = pd.DataFrame(comb, columns = ['alcohol', 'malic_acid', 'ash', 'alcalinity', 'magnesium', 'phenols', 'flavanoids', 'nonflavanoid_phenols',
                                  'proanthocyanins', 'color_intensity', 'hue', 'od280', 'proline', 'target'])


numerical_features = df.select_dtypes(['int64', 'float64']).columns.values
categorical_features = df.select_dtypes(['object']).columns.values

numerical_transform = MinMaxScaler(feature_range=(-1,1))
numerical_processed = numerical_transform.fit_transform(df[numerical_features])

df_processed = pd.get_dummies(df, columns= categorical_features)
df_processed.update(pd.DataFrame(numerical_processed, columns = numerical_features))
df_processed.head()


,alcohol,malic_acid,ash,alcalinity,magnesium,phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280,proline,target
0,0.684211,-0.616601,0.144385,-0.484536,0.239130,0.255172,0.147679,-0.433962,0.186120,-0.255973,-0.089431,0.941392,0.122682,-1.0
1,0.142105,-0.588933,-0.165775,-0.938144,-0.347826,0.151724,0.021097,-0.509434,-0.451104,-0.470990,-0.073171,0.560440,0.101284,-1.0
2,0.121053,-0.359684,0.401070,-0.175258,-0.326087,0.255172,0.223629,-0.358491,0.514196,-0.249147,-0.105691,0.391941,0.293866,-1.0
3,0.757895,-0.521739,0.219251,-0.360825,-0.065217,0.979310,0.329114,-0.584906,0.116719,0.112628,-0.382114,0.597070,0.714693,-1.0
4,0.163158,-0.268775,0.614973,0.072165,0.043478,0.255172,-0.008439,-0.018868,-0.110410,-0.481229,-0.089431,0.216117,-0.348074,-1.0


In [33]:
train, test = train_test_split(df_processed, train_size=0.7, random_state=1)

def create_model(layer1=3, layer2=3):
    model = keras.Sequential()
    model.add(Dense(layer1, input_shape=(13,)))
    model.add(Activation('sigmoid')) 
    model.add(Dense(layer2))
    model.add(Activation('sigmoid'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy']) 
    return model

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_model)
param_grid = {'layer1':[3, 5, 7], 'layer2':[3, 5]}
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(train.drop(columns=['target']), train['target'])
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

1/1 [==============================] - 0s 152ms/step - loss: 0.6929 - accuracy: 0.4400


4/4 [==============================] - 0s 2ms/step - loss: 0.6881 - accuracy: 0.4115
Best: 0.442667 using {'layer1': 5, 'layer2': 5}
